# Part 4b - Model testing locally

Instead of deploying the model to a Sagemaker endpoint where it runs all the time we can also just download the model from S3 and run it locally in our notebook. This is very similar to what we did when we ran the zero-shot model. However, in this case we will not have the luxury to have the pipeline pre- and post-process the data for us. So we will do it ourselves.

In [ ]:
import boto3
import sagemaker

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()
client = boto3.client('sagemaker')

print(f"IAM role arn used for running training: {role}")
print(f"S3 bucket used for storing artifacts: {sess.default_bucket()}")

We locate the S3 location of the model, download, and extract it

In [ ]:
training_job = client.list_training_jobs()['TrainingJobSummaries'][0]['TrainingJobName']
model_data = sess.describe_training_job(training_job)['ModelArtifacts']['S3ModelArtifacts']
model_data

In [ ]:
!mkdir model

In [ ]:
!aws s3 cp $model_data model/

In [ ]:
!tar -xf model/model.tar.gz -C model/

In [ ]:
!rm -r model/checkpoint*

Now we load the model into a model object so we can create summaries

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('./model/')

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained('./model/').to('cpu').eval()

In [ ]:
import pandas as pd
df_test = pd.read_csv('data/test.csv')
ref_summaries = list(df_test['summary'])
texts = list(df_test['text'])

Let's test it to see if it worked

In [ ]:
texts[0]

In [ ]:
input_ids = tokenizer(texts[0], truncation=True, padding='longest', return_tensors="pt").input_ids.to('cpu')

In [ ]:
output = model.generate(input_ids, max_length=20)
predictions = tokenizer.batch_decode(output, skip_special_tokens=True)

In [ ]:
predictions

Now let's do it for all texts

In [ ]:
candidate_summaries = []

for i, text in enumerate(texts):
    if i % 100 == 0:
        print(i)
    input_ids = tokenizer(text, truncation=True, padding='longest', return_tensors="pt").input_ids.to('cpu')
    output = model.generate(input_ids, max_length=20)
    predictions = tokenizer.batch_decode(output, skip_special_tokens=True)
    candidate_summaries.append(predictions[0])

In [ ]:
file = open("model-summaries.txt", "w")
for s in candidate_summaries:
    file.write(s + "\n")
file.close()

In [ ]:
from datasets import load_metric
metric = load_metric("rouge")

In [ ]:
def calc_rouge_scores(candidates, references):
    result = metric.compute(predictions=candidates, references=references, use_stemmer=True)
    result = {key: round(value.mid.fmeasure * 100, 1) for key, value in result.items()}
    return result

In [ ]:
calc_rouge_scores(candidate_summaries, ref_summaries)